In [88]:
#import packages
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import openpyxl

In [89]:
#import data
df_crime = pd.read_csv(r'/Users/samuelclark/Documents/UMBC Data Science /Machine learning/Final Projects/Data/MergedData_2017-2021.csv')
df_covid_ward = pd.read_excel(r'/Users/samuelclark/Documents/UMBC Data Science /Machine learning/Final Projects/Data/DC-COVID-19-Data-for-11-5-2021-2.xlsx'
                              , sheet_name = 'Total Cases by Ward', engine='openpyxl')
df_covid_death = pd.read_excel(r'/Users/samuelclark/Documents/UMBC Data Science /Machine learning/Final Projects/Data/DC-COVID-19-Data-for-11-5-2021-2.xlsx'
                               , sheet_name = 'Lives Lost by Ward', engine='openpyxl')
df_sick_police = pd.read_csv(r'/Users/samuelclark/Documents/UMBC Data Science /Machine learning/Final Projects/Data/daily_count_police_out_due_to_covid.csv')

In [90]:
#delete columns not needed from crime data
del df_crime['REPORT_DAT']
del df_crime['BLOCK']
del df_crime['YBLOCK']
del df_crime['XBLOCK']
#del df_crime['LATITUDE']
#del df_crime['LONGITUDE']
del df_crime['END_DATE']
del df_crime['METHOD']
del df_crime['DISTRICT']



df_crime = df_crime.rename(columns = {'START_DATE':'date'})

df_crime['date'] = pd.to_datetime(df_crime['date']).dt.date
df_crime['date'] = pd.to_datetime(df_crime['date'])
df_crime.dtypes
df_crime

,SHIFT,OFFENSE,WARD,LATITUDE,LONGITUDE,date
0,MIDNIGHT,THEFT F/AUTO,5.0,38.918707,-76.962533,2017-10-28
1,MIDNIGHT,ASSAULT W/DANGEROUS WEAPON,6.0,38.907643,-77.009053,2017-10-28
2,MIDNIGHT,THEFT F/AUTO,6.0,38.901323,-77.016914,2017-10-28
3,DAY,THEFT F/AUTO,6.0,38.883239,-77.018730,2017-10-27
4,DAY,THEFT/OTHER,5.0,38.920933,-76.997157,2017-10-28
...,...,...,...,...,...,...
151064,DAY,BURGLARY,5.0,38.913022,-76.975955,2021-02-04
151065,DAY,THEFT F/AUTO,7.0,38.874610,-76.967094,2021-02-04
151066,DAY,THEFT F/AUTO,5.0,38.951338,-76.991691,2021-02-03
151067,DAY,ASSAULT W/DANGEROUS WEAPON,8.0,38.852403,-76.982313,2021-02-04


In [91]:
df_crime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151069 entries, 0 to 151068
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   SHIFT      151069 non-null  object        
 1   OFFENSE    151069 non-null  object        
 2   WARD       147528 non-null  float64       
 3   LATITUDE   151069 non-null  float64       
 4   LONGITUDE  151069 non-null  float64       
 5   date       151064 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(3), object(2)
memory usage: 6.9+ MB


In [92]:
df_crime.isna().sum()

SHIFT           0
OFFENSE         0
WARD         3541
LATITUDE        0
LONGITUDE       0
date            5
dtype: int64

In [93]:
#drop records with null records
df_crime = df_crime.dropna(axis=0)
print(df_crime.isna().sum())
print(df_crime.shape)

SHIFT        0
OFFENSE      0
WARD         0
LATITUDE     0
LONGITUDE    0
date         0
dtype: int64
(147523, 6)


In [94]:
#covid case data
df_covid_ward = df_covid_ward.transpose()
df_covid_ward.reset_index(drop = True, inplace= True)
df_covid_ward.columns=('date', 'ward_1', 'ward_2', 'ward_3', 'ward_4', 'ward_5', 'ward_6'
                      , 'ward_7', 'ward_8', 'unknown')
df_covid_ward = df_covid_ward.iloc[1: , :]
df_covid_ward['date'] = pd.to_datetime(df_covid_ward['date'])
df_covid_ward['total_cases'] = (df_covid_ward['ward_1'] + df_covid_ward['ward_2'] + df_covid_ward['ward_3'] + df_covid_ward['ward_4'] + df_covid_ward['ward_5'] + df_covid_ward['ward_6'] +df_covid_ward['ward_7'] + df_covid_ward['ward_8'] +df_covid_ward['unknown'])
df_covid_ward['daily_cases'] = df_covid_ward['total_cases'].diff(1)

del df_covid_ward['ward_1']
del df_covid_ward['ward_2']
del df_covid_ward['ward_3']
del df_covid_ward['ward_4']
del df_covid_ward['ward_5']
del df_covid_ward['ward_6']
del df_covid_ward['ward_7']
del df_covid_ward['ward_8']
del df_covid_ward['unknown']

df_covid_ward.fillna(0, inplace=True)

df_covid_ward

,date,total_cases,daily_cases
1,2020-03-31,586,0
2,2020-04-01,653,67
3,2020-04-02,757,104
4,2020-04-03,902,145
5,2020-04-04,998,96
...,...,...,...
536,2021-11-01,64541,71
537,2021-11-02,64602,61
538,2021-11-03,64708,106
539,2021-11-04,64799,91


In [95]:
#death data
df_covid_death = df_covid_death.transpose()
df_covid_death.reset_index(drop = True, inplace= True)
df_covid_death.columns=('date', 'all', 'ward_1', 'ward_2', 'ward_3', 'ward_4', 'ward_5', 'ward_6'
                      , 'ward_7', 'ward_8', 'Experienced Homelessness', 'unknown', 'na')
df_covid_death = df_covid_death.iloc[1: , :]
df_covid_death['date'] = pd.to_datetime(df_covid_death['date'])
df_covid_death['daily_death'] = df_covid_death['all'].diff(1)

del df_covid_death['ward_1']
del df_covid_death['ward_2']
del df_covid_death['ward_3']
del df_covid_death['ward_4']
del df_covid_death['ward_5']
del df_covid_death['ward_6']
del df_covid_death['ward_7']
del df_covid_death['ward_8']
del df_covid_death['unknown']
del df_covid_death['na']
del df_covid_death['Experienced Homelessness']
del df_covid_death['all']

df_covid_death.fillna(0, inplace=True)

df_covid_death

,date,daily_death
1,2020-04-19,0
2,2020-04-20,7
3,2020-04-21,15
4,2020-04-22,12
5,2020-04-23,14
...,...,...
515,2021-11-01,2
516,2021-11-02,0
517,2021-11-03,0
518,2021-11-04,0


In [96]:
#sick police
df_sick_police = df_sick_police.rename(columns = {'DATE_REPORTED':'date'})
df_sick_police['date'] = pd.to_datetime(df_sick_police['date'])

df_sick_police.fillna(0, inplace=True)

df_sick_police

,date,OUT_POSITIVE_QUARANTINE_PSMPD
0,2020-03-07,0.0
1,2020-03-08,0.0
2,2020-03-09,0.0
3,2020-03-10,0.0
4,2020-03-11,0.0
...,...,...
589,2021-10-17,42.0
590,2021-10-18,43.0
591,2021-10-19,43.0
592,2021-10-20,43.0


In [97]:
#merge the dataframes together
merged = df_crime.merge(df_covid_ward, how='left', on = 'date').merge(df_covid_death, how='left', on = 'date').merge(df_sick_police, how='left', on = 'date')
merged

,SHIFT,OFFENSE,WARD,LATITUDE,LONGITUDE,date,total_cases,daily_cases,daily_death,OUT_POSITIVE_QUARANTINE_PSMPD
0,MIDNIGHT,THEFT F/AUTO,5.0,38.918707,-76.962533,2017-10-28,NaN,NaN,NaN,NaN
1,MIDNIGHT,ASSAULT W/DANGEROUS WEAPON,6.0,38.907643,-77.009053,2017-10-28,NaN,NaN,NaN,NaN
2,MIDNIGHT,THEFT F/AUTO,6.0,38.901323,-77.016914,2017-10-28,NaN,NaN,NaN,NaN
3,DAY,THEFT F/AUTO,6.0,38.883239,-77.018730,2017-10-27,NaN,NaN,NaN,NaN
4,DAY,THEFT/OTHER,5.0,38.920933,-76.997157,2017-10-28,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
147518,DAY,BURGLARY,5.0,38.913022,-76.975955,2021-02-04,37634.0,269.0,8.0,104.0
147519,DAY,THEFT F/AUTO,7.0,38.874610,-76.967094,2021-02-04,37634.0,269.0,8.0,104.0
147520,DAY,THEFT F/AUTO,5.0,38.951338,-76.991691,2021-02-03,37365.0,166.0,6.0,106.0
147521,DAY,ASSAULT W/DANGEROUS WEAPON,8.0,38.852403,-76.982313,2021-02-04,37634.0,269.0,8.0,104.0


In [98]:
#getting weekday dummies
merged['weekday'] = merged['date'].dt.dayofweek
dummy0 = pd.get_dummies(merged['weekday'])
dummy0.columns = ['MO', 'TU', 'WE', 'TH', 'FR', 'SA', 'SU']
merged = pd.concat([merged, dummy0] , axis = 1)
del merged['weekday']

#get month dummies
merged['month'] = merged['date'].dt.month
dummy1 = pd.get_dummies(merged['month'])
dummy1.columns = ['JAN', 'FEB','MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP',
                 'OCT', 'NOV', 'DEC']              
merged = pd.concat([merged, dummy1] , axis = 1)
del merged['month']

#ward dummies
dummy2 = pd.get_dummies(merged['WARD'])
dummy2.columns = ['ward_1', 'ward_2','ward_3', 'ward_4', 'ward_5', 'ward_6', 'ward_7', 'ward_8']              
merged = pd.concat([merged, dummy2] , axis = 1)

#offense dummies
dummy3 = pd.get_dummies(merged['OFFENSE'])
merged = pd.concat([merged, dummy3] , axis = 1)

#shift dummies
dummies4 = pd.get_dummies(merged.SHIFT)
merged = pd.concat([merged, dummies4], axis=1)


merged.columns

Index(['SHIFT', 'OFFENSE', 'WARD', 'LATITUDE', 'LONGITUDE', 'date',
       'total_cases', 'daily_cases', 'daily_death',
       'OUT_POSITIVE_QUARANTINE_PSMPD', 'MO', 'TU', 'WE', 'TH', 'FR', 'SA',
       'SU', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP',
       'OCT', 'NOV', 'DEC', 'ward_1', 'ward_2', 'ward_3', 'ward_4', 'ward_5',
       'ward_6', 'ward_7', 'ward_8', 'ARSON', 'ASSAULT W/DANGEROUS WEAPON',
       'BURGLARY', 'HOMICIDE', 'MOTOR VEHICLE THEFT', 'ROBBERY', 'SEX ABUSE',
       'THEFT F/AUTO', 'THEFT/OTHER', 'DAY', 'EVENING', 'MIDNIGHT'],
      dtype='object')

In [99]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 147523 entries, 0 to 147522
Data columns (total 49 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   SHIFT                          147523 non-null  object        
 1   OFFENSE                        147523 non-null  object        
 2   WARD                           147523 non-null  float64       
 3   LATITUDE                       147523 non-null  float64       
 4   LONGITUDE                      147523 non-null  float64       
 5   date                           147523 non-null  datetime64[ns]
 6   total_cases                    38223 non-null   float64       
 7   daily_cases                    38223 non-null   float64       
 8   daily_death                    37115 non-null   float64       
 9   OUT_POSITIVE_QUARANTINE_PSMPD  43556 non-null   float64       
 10  MO                             147523 non-null  uint8         
 11  

In [100]:
#replace nan with 0
merged.fillna(0, inplace=True)

merged

,SHIFT,OFFENSE,WARD,LATITUDE,LONGITUDE,date,total_cases,daily_cases,daily_death,OUT_POSITIVE_QUARANTINE_PSMPD,...,BURGLARY,HOMICIDE,MOTOR VEHICLE THEFT,ROBBERY,SEX ABUSE,THEFT F/AUTO,THEFT/OTHER,DAY,EVENING,MIDNIGHT
0,MIDNIGHT,THEFT F/AUTO,5.0,38.918707,-76.962533,2017-10-28,0.0,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0,0,1
1,MIDNIGHT,ASSAULT W/DANGEROUS WEAPON,6.0,38.907643,-77.009053,2017-10-28,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1
2,MIDNIGHT,THEFT F/AUTO,6.0,38.901323,-77.016914,2017-10-28,0.0,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0,0,1
3,DAY,THEFT F/AUTO,6.0,38.883239,-77.018730,2017-10-27,0.0,0.0,0.0,0.0,...,0,0,0,0,0,1,0,1,0,0
4,DAY,THEFT/OTHER,5.0,38.920933,-76.997157,2017-10-28,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147518,DAY,BURGLARY,5.0,38.913022,-76.975955,2021-02-04,37634.0,269.0,8.0,104.0,...,1,0,0,0,0,0,0,1,0,0
147519,DAY,THEFT F/AUTO,7.0,38.874610,-76.967094,2021-02-04,37634.0,269.0,8.0,104.0,...,0,0,0,0,0,1,0,1,0,0
147520,DAY,THEFT F/AUTO,5.0,38.951338,-76.991691,2021-02-03,37365.0,166.0,6.0,106.0,...,0,0,0,0,0,1,0,1,0,0
147521,DAY,ASSAULT W/DANGEROUS WEAPON,8.0,38.852403,-76.982313,2021-02-04,37634.0,269.0,8.0,104.0,...,0,0,0,0,0,0,0,1,0,0


In [101]:
#split into precovid/during covid
merged.to_csv(r'/Users/samuelclark/Documents/UMBC Data Science /Machine learning/Final Projects/Data/merged.csv')
